# 🥋 Lekcja 26: Dynamic Control Flow (Python wewnątrz sieci)

W PyTorch metoda `forward()` to zwykły kod Python.
Możesz używać `if`, `for`, `while`, a nawet `print()`.

**Jak to działa?**
Graf obliczeniowy nie jest budowany raz na zawsze na początku.
Jest budowany **od nowa przy każdym przejściu `forward`**.

*   Jeśli w Iteracji 1 wejdziesz do `if`: Graf zawiera gałąź A.
*   Jeśli w Iteracji 2 wejdziesz do `else`: Graf zawiera gałąź B.

To pozwala na budowanie **Dynamicznych Sieci Neuronowych**.

In [1]:
import torch
import torch.nn as nn

# Konfiguracja
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Urządzenie: {DEVICE}")

Urządzenie: cuda


## Eksperyment 1: Warunek `if` (Data-Dependent Control Flow)

Stworzymy sieć "Dziwaczną".
*   Jeśli suma wejścia jest dodatnia -> Użyj warstwy `fc_pos`.
*   Jeśli suma wejścia jest ujemna -> Użyj warstwy `fc_neg`.

Silnik Autograd musi poradzić sobie z tym, że w jednej iteracji używamy jednych wag, a w drugiej innych.

In [6]:
class WeirdNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc_pos = nn.Linear(10, 1) # Dla liczb dodatnich
        self.fc_neg = nn.Linear(10, 1) # Dla liczb ujemnych
        
        # Inicjalizacja dla rozróżnienia
        nn.init.constant_(self.fc_pos.weight, 1.0)
        nn.init.constant_(self.fc_neg.weight, -1.0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # LOGIKA PYTHONOWA W ŚRODKU SIECI
        s = x.sum()
        
        if s > 0:
            print("   -> Ścieżka POZYTYWNA")
            x = self.fc_pos(x)
        else:
            print("   -> Ścieżka NEGATYWNA")
            x = self.fc_neg(x)
            
        return x

model = WeirdNet().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Test 1: Dane dodatnie
input_pos = torch.ones(1, 10).to(DEVICE)
out_pos = model(input_pos)
out_pos.backward()

print(f"Gradient fc_pos: {model.fc_pos.weight.grad[0,0]}") # Powinien być (1.0)
print(f"Gradient fc_neg: {model.fc_neg.weight.grad}")      # Powinien być None (nieużywany)

# Czyścimy
optimizer.zero_grad()

# Test 2: Dane ujemne
print("-" * 20)
input_neg = -torch.ones(1, 10).to(DEVICE)
out_neg = model(input_neg)
out_neg.backward()

print(f"Gradient fc_pos: {model.fc_pos.weight.grad}")      # Teraz to jest None/0
print(f"Gradient fc_neg: {model.fc_neg.weight.grad[0,0]}") # Teraz to ma wartość (-1.0)

   -> Ścieżka POZYTYWNA
Gradient fc_pos: 1.0
Gradient fc_neg: None
--------------------
   -> Ścieżka NEGATYWNA
Gradient fc_pos: None
Gradient fc_neg: -1.0


## Eksperyment 2: Pętla `for` (Weight Sharing w czasie)

To jest fundament sieci RNN.
Używamy **tej samej warstwy** wielokrotnie w pętli.

PyTorch jest na tyle sprytny, że wie: *"Użyłeś tej warstwy 5 razy. Podczas `backward` muszę zsumować gradienty z tych 5 użyć"*.

In [7]:
class LoopNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(10, 10) # Jedna warstwa
        
    def forward(self, x: torch.Tensor, steps: int) -> torch.Tensor:
        # Dynamiczna pętla - liczba kroków zależy od argumentu wywołania!
        for i in range(steps):
            x = self.fc(x)
            # Możemy nawet zrobić coś szalonego:
            if x.mean() > 100:
                print(f"   (Przerwanie pętli w kroku {i} - wybuch wartości)")
                break
        return x

loop_model = LoopNet().to(DEVICE)

# Uruchamiamy na 3 kroki
x = torch.randn(1, 10).to(DEVICE)
out = loop_model(x, steps=3)
out.sum().backward()

print("Model z pętlą działa.")
print(f"Gradient wagi (istnieje?): {loop_model.fc.weight.grad is not None}")

Model z pętlą działa.
Gradient wagi (istnieje?): True


## Pułapka: Wydajność i Eksport

Dynamiczne grafy są super do debugowania i badań. Ale mają wadę na produkcji.

1.  **Brak optymalizacji:** Kompilator nie wie, co się stanie (czy wejdziemy w `if`, ile razy obróci się `for`). Trudno złączyć operacje (Operator Fusion).
2.  **Eksport (ONNX):** ONNX woli statyczne grafy. Eksport modelu z `if x.sum() > 0` może być trudny lub niemożliwy (trzeba używać `torch.jit.script` zamiast `trace`).

In [10]:
# 1. Kompilacja (Scripting)
# Analizuje AST (drzewo składniowe) Pythona i kompiluje logikę.
scripted_model = torch.jit.script(model)

# 2. Dowód 1: Inspekcja Kodu
# Wyświetlamy to, jak TorchScript "zrozumiał" nasz model.
# Powinieneś zobaczyć instrukcję: "if bool(torch.gt(s, 0.)):"
print("\n[Dowód 1] Skompilowany kod (IR):")
print(scripted_model.code)

# 3. Dowód 2: Test Numeryczny
print("[Dowód 2] Uruchomienie na danych ujemnych...")
# Wejście: same -1.
# Ścieżka Positive (Waga 1.0): -1 * 1 = -1
# Ścieżka Negative (Waga -1.0): -1 * -1 = 1 (Tego oczekujemy)

out_jit = scripted_model(input_neg)
mean_val = out_jit.mean().item()

print(f"Średnia wartość wyniku: {mean_val}")

if mean_val > 0:
    print("✅ SUKCES: Wynik dodatni. Model poprawnie wybrał ścieżkę 'else' (fc_neg).")
else:
    print("❌ BŁĄD: Wynik ujemny. Model błędnie poszedł ścieżką 'if' (fc_pos).")


[Dowód 1] Skompilowany kod (IR):
def forward(self,
    x: Tensor) -> Tensor:
  s = torch.sum(x)
  if bool(torch.gt(s, 0)):
    print(CONSTANTS.c0)
    fc_pos = self.fc_pos
    x0 = (fc_pos).forward(x, )
  else:
    print(CONSTANTS.c1)
    fc_neg = self.fc_neg
    x0 = (fc_neg).forward(x, )
  return x0

[Dowód 2] Uruchomienie na danych ujemnych...
   -> Ścieżka NEGATYWNA
Średnia wartość wyniku: 10.278833389282227
✅ SUKCES: Wynik dodatni. Model poprawnie wybrał ścieżkę 'else' (fc_neg).


## 🥋 Black Belt Summary

1.  **Define-by-Run:** PyTorch buduje graf w locie. To pozwala na używanie natywnego Pythona (`if`, `for`, `print`).
2.  **Gradienty:** Autograd automatycznie radzi sobie z warunkowością. Nieużywane gałęzie nie dostają gradientów. Wielokrotnie używane warstwy (pętle) akumulują gradienty.
3.  **Cena:** Dynamiczne grafy są trudniejsze do zoptymalizowania (`torch.compile`) i wyeksportowania (`ONNX`).
    *   Jeśli musisz eksportować logikę `if`, używaj **`torch.jit.script`**, a nie `trace`.